Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user input. In these types of chains, there is an “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

In [1]:
//Import the Ollama Large Language Model
import { Ollama } from "npm:@langchain/community/llms/ollama";

//Import the agent executor module
import { initializeAgentExecutorWithOptions } from "npm:langchain/agents";

//Import the SerpAPI and Calculator tools
import { SerpAPI } from "npm:@langchain/community/tools/serpapi";

import { Calculator } from "npm:langchain/tools/calculator"


In [2]:
import * as serpapi from "npm:serpapi"

In [3]:
import * as dotenv from "npm:dotenv";
const env = dotenv.config();

In [4]:
export const run = async (input) => {

      //Instantiante the OpenAI model 
      //Pass the "temperature" parameter which controls the RANDOMNESS of the model's output. A lower temperature will result in more predictable output, while a higher temperature will result in more random output. The temperature parameter is set between 0 and 1, with 0 being the most predictable and 1 being the most random
      const model = new Ollama({ temperature: 0 });

      //Create a list of the instatiated tools
      const tools = [new SerpAPI(env.SERPAPI_API_KEY, {
      location: "Austin,Texas,United States",
      hl: "en",
      gl: "us",
    }), new Calculator()];

      //Construct an agent from an LLM and a list of tools
      //"zero-shot-react-description" tells the agent to use the ReAct framework to determine which tool to use. The ReAct framework determines which tool to use based solely on the tool’s description. Any number of tools can be provided. This agent requires that a description is provided for each tool.
      const executor = await initializeAgentExecutorWithOptions(tools, model, {
    agentType: "chat-conversational-react-description",
    verbose: true,
  });
      console.log("Loaded agent.");

      //Specify the prompt
      console.log(`Executing with input "${input}"...`);

      //Run the agent
      const result = await executor.call({ input });

      console.log(`Got output ${result.output}`);
  };

In [5]:
const input =
      "Hi, i am bob";

await run(input);

Loaded agent.
Executing with input "Hi, i am bob"...
[chain/start] [1:chain:AgentExecutor] Entering Chain run with input: {
  "input": "Hi, i am bob",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input: {
  "input": "Hi, i am bob",
  "chat_history": [],
  "agent_scratchpad": [],
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:Ollama] Entering LLM run with input: {
  "prompts": [
    "System: Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly

In [6]:
const input =
      "Who is Beyonce's husband?" +
      " What is his current age raised to the 0.23 power?";

await run(input);

Loaded agent.
Executing with input "Who is Beyonce's husband? What is his current age raised to the 0.23 power?"...
[chain/start] [1:chain:AgentExecutor] Entering Chain run with input: {
  "input": "Who is Beyonce's husband? What is his current age raised to the 0.23 power?",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input: {
  "input": "Who is Beyonce's husband? What is his current age raised to the 0.23 power?",
  "chat_history": [],
  "agent_scratchpad": [],
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:Ollama] Entering LLM run with input: {
  "prompts": [
    "System: Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text 

Error: Failed to parse. Text: "```json
{{
    "action": "Search",
    "action_input": "Beyonce's husband"
}}
```
Or, if you want to respond directly and conversationally:

{{
    "action": "Final Answer",
    "action_input": "Jay-Z is Beyonce's husband. According to recent reports, he is currently 53 years old."
}}". Error: SyntaxError: Expected property name or '}' in JSON at position 1 (line 1 column 2)

In [8]:
import { AgentExecutor, createReactAgent } from "npm:langchain/agents";
import { pull } from "npm:langchain/hub";
import type { PromptTemplate } from "npm:@langchain/core/prompts";

// Get the prompt to use - you can modify this!
// If you want to see the prompt in full, you can at:
// https://smith.langchain.com/hub/hwchase17/react
const prompt = await pull<PromptTemplate>("hwchase17/react");

const llm = new Ollama({
  temperature: 0,
});

const tools = [new SerpAPI(env.SERPAPI_API_KEY, {
      location: "Austin,Texas,United States",
      hl: "en",
      gl: "us",
    }), new Calculator()];

const agent = await createReactAgent({
  llm,
  tools,
  prompt
});

In [10]:
const agentExecutor = new AgentExecutor({
  agent,
  tools,
});

const result = await agentExecutor.call({
  input: "what is LangChain?",
});

console.log(result);


{
  input: "what is LangChain?",
  output: "Agent stopped due to max iterations."
}
